# Experimenting with Cleaning, Clustering & Summarization Pipelines

### To do (technical)
- Implement date windows on my corpus loader function

In [1]:
import os
import re
import json

import numpy as np
import pandas as pd
import networkx as nx

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

import lib.helper as helper
import lib.embedding_models as reps

from importlib import reload

%matplotlib inline

# Useful flatten function from Alex Martelli on https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
flatten = lambda l: [item for sublist in l for item in sublist]

## 1.  Retrieve Corpus

The corpus is being scraped by the "run_news_scrapes.py" script (and windows task scheduler) every 12 hours, a bit past midday and a bit past midnight.

The "bing" corpus are news titles and text extracts gotten from the bing news search API, using a few Home Office - related keywords.

The "RSS" corpus is plugged directly into a number of RSS feeds for world news sites and local british news sites, with no filters for news story types or subjects applied.

In [2]:
# Should be same path for all my PC's, it's where each scrape goes as a separate json file.
storage_path = "D:/Dropbox/news_crow/scrape_results"

# "bing" is targeted news search corpus, "RSS" is from specific world and local news feeds.
corpus_type = "RSS"

# There's a helper function to go find and drag out the various JSON files created by the scrapers.
corpus = helper.load_clean_corpus(storage_path, corpus_type)

# Make sure after cleaning etc it's indexed from 0
corpus.reset_index(inplace=True)
corpus.index.name = "node"

# Subset to latest half to see if it'll run in memory
corpus = corpus.iloc[-60000:,]

# See how it turned out
print(corpus.shape)
corpus.head()

Total files: 485
9.9 percent of files read.
19.8 percent of files read.
29.7 percent of files read.
39.6 percent of files read.
49.5 percent of files read.
59.4 percent of files read.
69.3 percent of files read.
79.2 percent of files read.
89.1 percent of files read.
99.0 percent of files read.
(60000, 9)


,index,title,summary,date,link,source_url,retrieval_timestamp,origin,clean_text
node,,,,,,,,,
60192,598015,Rudy Giuliani sidekick Lev Parnas surfed porn ...,A phone that Lev Parnas turned over to the Hou...,"Wed, 29 Jan 2020 23:00:37 GMT",https://www.dailymail.co.uk/news/article-79444...,https://www.dailymail.co.uk/news/articles.rss,2020-01-30 00:23:13.270518,rss_feed,Rudy Giuliani sidekick Lev Parnas surfed porn ...
60193,598016,Mallacoota family break down in tears on the T...,The Nelson-Parthenides family fled their home ...,"Wed, 29 Jan 2020 22:51:28 GMT",https://www.dailymail.co.uk/news/article-79446...,https://www.dailymail.co.uk/news/articles.rss,2020-01-30 00:23:13.270527,rss_feed,Mallacoota family break down in tears on the T...
60194,598017,"Michigan man, 68, who served 24 of a 60-year p...","Michael Thompson, 68, is serving a 60-year pri...","Wed, 29 Jan 2020 22:36:57 GMT",https://www.dailymail.co.uk/news/article-79443...,https://www.dailymail.co.uk/news/articles.rss,2020-01-30 00:23:13.270535,rss_feed,"Michigan man, 68, who served 24 of a 60-year p..."
60195,598018,Australian in Wuhan says his mother is terrifi...,Daniel Ou Yang is an Australian in the Chinese...,"Wed, 29 Jan 2020 22:32:38 GMT",https://www.dailymail.co.uk/news/article-79448...,https://www.dailymail.co.uk/news/articles.rss,2020-01-30 00:23:13.270544,rss_feed,Australian in Wuhan says his mother is terrifi...
60196,598019,Five and ten cent coins to disappear within a ...,Silver five and 10 cent coins were introduced ...,"Wed, 29 Jan 2020 22:32:27 GMT",https://www.dailymail.co.uk/news/article-79445...,https://www.dailymail.co.uk/news/articles.rss,2020-01-30 00:23:13.270552,rss_feed,Five and ten cent coins to disappear within a ...


## 2. Use Detected Nouns to create a Graph Representation

In [3]:
# Generate the text representation
model = reps.NounAdjacencyModel(list(corpus['clean_text']), list(corpus['clean_text']))

# Tabulate for convenience
nouns_df = model.table.copy()
nouns_df.head()

,Ambulance,seeapos,Maple,Shore,talkapos,Antonio_Guterres,Beers,Video,Wuhan_Hubei,A20,...,Schumerapos;s,apos;disturbing,Champs,Blair,Inhumane,FTSE_Britain,Hindley,Democrat_Joe,Teague,Telarah
"Rudy Giuliani sidekick Lev Parnas surfed porn and hook-up websites, phone he turned over reveals. A phone that Lev Parnas turned over to the House containing voluminous materials about the Ukraine affair also includes searches of porn and hookup sites, according to a report.",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Mallacoota family break down in tears on the Today Show after they're given a car after bushfire. The Nelson-Parthenides family fled their home in Mallacoota on December 2 - just days before the skies turned an eerie red as a ferocious blaze tore through the region.,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Michigan man, 68, who served 24 of a 60-year prison sentence for selling pot, pleads for clemency. Michael Thompson, 68, is serving a 60-year prison sentence in Michigan for selling marijuana to an informant on Wednesday asked for a clemency for a second time in the hopes of an earlier release",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Australian in Wuhan says his mother is terrified he may be quarantined over coronavirus. Daniel Ou Yang is an Australian in the Chinese city of Wuhan and fearful of his uncertain future.,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Five and ten cent coins to disappear within a decade - Australian Mint demand plummet. Silver five and 10 cent coins were introduced in Australia in 166, and 46million of them were minted in 2006.",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Drop any noun/noun phrase containing one of the search terms, then create an adjacency matrix

#### Drop any noun/phrase occuring too infrequently

In [4]:
# Retrive the set of search terms used for Bing, so we can remove them before
# clustering.
with open("D:/Dropbox/news_crow/scrape_settings.json", "r") as f:
    scrape_config = json.load(f)

search_terms = scrape_config['disaster_search_list']
search_terms = re.sub(r"[^0-9A-Za-z ]", "", " ".join(search_terms)).lower().split()
search_terms = set(search_terms)

In [5]:
# Get X most common nouns WHY NOT USE TFIDF HERE?
nouns_to_keep = list(nouns_df.\
                    sum(axis=0).\
                    sort_values(ascending=False).\
                    index)

# Cut out any nouns containing the original search terms
#nouns_to_keep = [noun for noun in nouns_to_keep if sum([term in noun for term in search_terms]) == 0]

# Keep only most common
nouns_to_keep = nouns_to_keep[:2000]

# Subset nouns dataframe
nouns_df = nouns_df[nouns_to_keep]

print(nouns_df.shape)

(60000, 2000)


In [ ]:
embeddings = np.asarray(nouns_df)
adjacency = np.dot(embeddings, embeddings.T)
print(np.max(adjacency))

In [ ]:
# If the "lower" limit is 1, the graph has so many edges it eats ALL the memory of my desktop, even
# with just 500-ish stories to process.
upper = 100
lower = 3
G = nx.Graph()
rows, cols = np.where((upper >= adjacency) & (adjacency >= lower))
weights = [float(adjacency[rows[i], cols[i]]) for i in range(len(rows))]
edges = zip(rows.tolist(), cols.tolist(), weights)
G.add_weighted_edges_from(edges)

# Simplify; remove self-edges - not sure if needed?
G.remove_edges_from(nx.selfloop_edges(G))

In [ ]:
G.number_of_edges()

12596 to beat

## 3c.  Try CDLIB

In [ ]:
import cdlib
from cdlib import algorithms
from cdlib import evaluation

In [ ]:
# Simple (flat) clustering
lp_coms = algorithms.label_propagation(G)

# Traditional (easy) community detection
louvain_coms = algorithms.louvain(G)

In [ ]:
# This result implies that the two methods have come to very similar conclusions...
# This function apparently isn't defined for overlapping communities
evaluation.normalized_mutual_information(lp_coms, louvain_coms)

In [ ]:
# Build dict of node-to-cluster lookup
community_lookup = {}
for comm_index, members in enumerate(louvain_coms.communities):
    for member in members:
        community_lookup[member] = comm_index

# Add cluster to DF.  If node not in cluster, assign -1 (outlier)
corpus['node'] = corpus.index
corpus['cluster'] = corpus['node'].apply(lambda x: community_lookup.get(x, -1))
corpus[['clean_text', 'cluster']].head(10)

# If cluster is smaller than minimum limit, designate as outlier
cs_lookup = corpus['cluster'].value_counts().to_dict()
corpus['cluster'] = corpus['cluster'].apply(lambda x: -1 if (cs_lookup[x] < 5) else x)

In [ ]:
# What percentage are now classed as outliers?
100.0 * sum(corpus['cluster']==-1) / corpus.shape[0]

In [ ]:
# How many unique clusters after all this?  (minus one for outliers)
len(pd.unique(corpus['cluster']))

In [ ]:
corpus.to_csv("working/RSS_clustered_louvain.csv")

In [ ]:
#bigclam_coms.communities

In [ ]:
#bigclam_coms.average_internal_degree()

In [ ]:
#bigclam_coms.newman_girvan_modularity()

## 3.  Create (overlapping) clusters using Maximal Cliques
Idea from the docs, explanation at https://en.wikipedia.org/wiki/Clique_(graph_theory)
Expanded using k-clique-communities REF FIND PAPER

In [ ]:
c = list(nx.algorithms.community.kclique.k_clique_communities(G, 4))
cliques = [(len(x), x) for x in c]

In [ ]:
cliques

In [ ]:
cliques_df = pd.DataFrame({"nodes_list": [x[1] for x in cliques],
                           "clique_size": [x[0] for x in cliques]}).\
                    sort_values("clique_size", ascending=False).\
                    reset_index()

cliques_df = cliques_df[(cliques_df['clique_size'] >= 3) & (cliques_df['clique_size'] <=100)]

In [ ]:
cliques_df

In [ ]:
cliqued = set(flatten(list(cliques_df['nodes_list'])))
len(cliqued)

In [ ]:
# Flatten the cliques DF into long format
flattened = {"cluster_index":[], "node":[]}

for index, row in cliques_df.iterrows():
    for node in row["nodes_list"]:
        flattened["cluster_index"].append(index)
        flattened["node"].append(node)
        

partition_df = pd.DataFrame(flattened)

# Create a single string variable (";" separated) to record all clusters/cliques a single record belongs in
partition_df["cluster"] = partition_df.\
                          groupby("node")["cluster_index"].\
                          transform(lambda x: ";".join([str(i) for i in x if type(i)==int]))

# Clean up, set index of this and corpus so the two DF's can be joined with little effort
partition_df = partition_df[["node", "cluster"]].\
               drop_duplicates(["node", "cluster"], keep="first").\
               set_index("node")

corpus.drop(["cluster", "node"], axis=1).join(partition_df).\
       to_csv("working/RSS_clustered_cliques.csv")

### The below attempts overlapping community detection but can only run on connected graphs, think this is an implicit restriction of the algorithm logic.

In [ ]:
# Get all connected components (will become less of an issue as graph size increases)
ccs = [(len(x), x) for x in nx.connected_components(G)]

# Sort by size (largest first)
ccs.sort(key = lambda x: x[0], reverse=True)

# Extract largest connected sub-graph
connected_sub = G.subgraph(ccs[0][1])

# re-index nodes from zero to maintain compatibility with CDLIB (sub-dependency, Karate)
# Will need to reverse this indexing when matching assigned clusters back to data
node_relabel_dict = {val: i for i, val in enumerate(list(connected_sub.nodes))}

connected_sub = nx.relabel_nodes(connected_sub, node_relabel_dict)

# Fire algo!
bigclam_coms = algorithms.big_clam(connected_sub)
#leiden_coms = algorithms.leiden(connected_sub)

In [ ]:
bigclam_coms.communities

In [ ]:
# Build dict of node-to-cluster lookup
community_lookup = {}
for comm_index, members in enumerate(bigclam_coms.communities):
    for member in members:
        community_lookup[member] = community_lookup.get(member, []) + [comm_index]

# Add cluster to DF.  If node not in cluster, assign -1 (outlier)
corpus['node'] = corpus.index
corpus['cluster'] = corpus['node'].apply(lambda x: community_lookup.get(x, [-1]))
corpus[['clean_text', 'cluster']].head(10)

# If cluster is smaller than minimum limit, designate as outlier
cs_lookup = corpus['cluster'].value_counts().to_dict()
corpus['cluster'] = corpus['cluster'].apply(lambda x: -1 if (cs_lookup[x] < 5) else x)

In [ ]:
# What percentage are now classed as outliers?
100.0 * sum(corpus['cluster']==-1) / corpus.shape[0]

In [ ]:
# How many unique clusters after all this?  (minus one for outliers)
len(pd.unique(corpus['cluster']))

In [ ]:
corpus.to_csv("working/disaster_clustered_bigclam.csv")